In [50]:
from transformers import HerbertTokenizer, RobertaModel
import torch
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

In [51]:
torch.cuda.is_available()

True

In [52]:
tokenizer = HerbertTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")
model = RobertaModel.from_pretrained("allegro/herbert-klej-cased-v1")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLMTokenizer'. 
The class this function is called from is 'HerbertTokenizer'.


In [53]:
df = pd.read_csv('../dataset/dataset.csv', sep='$', encoding='utf-8')

# Split 

In [54]:
X = df['user_comment']
y = df['user_rate']
seed = 32
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=seed)

In [55]:
y_train = y_train.to_numpy()
y_valid = y_valid.to_numpy()

Save the test part for later

In [56]:
X_test.to_csv('test_input.csv', sep='$', encoding='utf-8')
y_test.to_csv('test_target.csv', sep='$', encoding='utf-8')

# Tokenization

In [57]:
def pre_parse_dataset(input):
    encoded_corpus = tokenizer(text=input, add_special_tokens=True, padding='max_length', truncation='longest_first', max_length=256, return_attention_mask=True)
    return np.array(encoded_corpus['input_ids']), np.array(encoded_corpus['attention_mask'])

In [58]:
training_input_id, training_attention_mask = pre_parse_dataset(X_train.tolist())
valid_input_id, valid_attention_mask = pre_parse_dataset(X_valid.tolist())

Consider rescaling the target values

# Dataset preparation for pytorch

In [59]:

def prepare_dataloader(inputs, masks, labels, batch_size):
    input_tensor = torch.tensor(inputs)
    mask_tensor = torch.tensor(masks)
    labels_tensor = torch.tensor(labels)
    dataset = TensorDataset(input_tensor, mask_tensor, labels_tensor)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

train_dataloader = prepare_dataloader(training_input_id, training_attention_mask, y_train, 32)
validation_dataloader = prepare_dataloader(valid_input_id, valid_attention_mask, y_valid, 32)